In [ ]:
import xarray as xr
import rioxarray

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def cdf(data):
    # compute cdf values for a 1D array of data
    data = data[~np.isnan(data)] # remove NaN values
    x = np.sort(data) #sort data
    y = 1. * np.arange(len(data)) / (len(data) - 1) #calculate CDF values
    return x, y

In [ ]:
base_filepath = './Data/NoahMP/'

In [ ]:
# read in NoahMP GDAS February mean SWE geotiff
swe_noahmp_gdas_febmean_mm = xr.open_rasterio(base_filepath + 'swe_noahmp_gdas_febmean_mm.tif').squeeze()

# replace nodata values (-1, or anything negative) with NaNs
swe_noahmp_gdas_febmean_mm = swe_noahmp_gdas_febmean_mm.where(swe_noahmp_gdas_febmean_mm >= 0)

# compute NoahMP GDAS CDF
gdas_cdf_x, gdas_cdf_y = cdf(swe_noahmp_gdas_febmean_mm.values.ravel())

# convert to a pandas dataframe
gdas_cdf_df = pd.DataFrame(data={'cdf_x': gdas_cdf_x, 'cdf_y': gdas_cdf_y})

# export CDF values to a csv file
gdas_cdf_df.to_csv('swe_noahmp_gdas_febmean_mm_CDF.csv')

In [ ]:
# read in NoahMP ECMWF February mean SWE geotiff
swe_noahmp_ecmwf_febmean_mm = xr.open_rasterio(base_filepath + 'swe_noahmp_ecmwf_febmean_mm.tif').squeeze()

# replace nodata values (-1, or anything negative) with NaNs
swe_noahmp_ecmwf_febmean_mm = swe_noahmp_ecmwf_febmean_mm.where(swe_noahmp_ecmwf_febmean_mm >= 0)

# compute NoahMP ECMWF CDF
ecmwf_cdf_x, ecmwf_cdf_y = cdf(swe_noahmp_ecmwf_febmean_mm.values.ravel())

# convert to a pandas dataframe
ecmwf_cdf_df = pd.DataFrame(data={'cdf_x': ecmwf_cdf_x, 'cdf_y': ecmwf_cdf_y})

# export CDF values to a csv file
ecmwf_cdf_df.to_csv('swe_noahmp_ecmwf_febmean_mm_CDF.csv')

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(15,4),tight_layout=True,gridspec_kw={'width_ratios': [1, 1, 1]})
[ax1, ax2, ax3] = ax.ravel()

### Map plots ###

# NoahMP GDAS map
swe_noahmp_gdas_febmean_mm.plot(ax=ax1, vmin=0, vmax=200, cmap='Blues', cbar_kwargs={'label': 'SWE (mm)'})

# NoahMP ECMWF map
swe_noahmp_ecmwf_febmean_mm.plot(ax=ax2, vmin=0, vmax=200, cmap='Blues', cbar_kwargs={'label': 'SWE (mm)'})

# format map plots
ax1.set_title('NoahMP GDAS\nMean February SWE')
ax2.set_title('NoahMP ECMWF\nMean February SWE')
for this_ax in [ax1, ax2]:
    this_ax.set_ylabel('Longitude')
    this_ax.set_xlabel('Latitude')

    
### CDF plot ###

# NoahMP GDAS CDF
ax3.plot(gdas_cdf_x, gdas_cdf_y, '-', label='NoahMP GDAS')

# NoahMP ECMWF CDF
ax3.plot(ecmwf_cdf_x, ecmwf_cdf_y, '--', label='NoahMP ECMWF')

# format CDF plot
ax3.set_xlim((-5,500))
ax3.set_title('CDF of Mean February SWE')
ax3.set_ylabel('Cumulative Frequency')
ax3.set_xlabel('Mean February SWE (mm)')
ax3.legend(loc='lower right');

# save figure
plt.savefig('NoahMP_FebMeanSWE.jpg')